In [4]:
import ee
import sys
print(sys.path)

from dotenv import load_dotenv

load_dotenv()


['/usr/src/app/notebooks', '/usr/src/app', '/usr/src/app/gee_lib', '/usr/src/app/ex_ante_lib', '/usr/src/app/forestry_carbon_arr', '/usr/src/app/fastapi-gee-service', '/usr/src/app/eemont-osi', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/opt/venv/lib/python3.12/site-packages', '__editable__.eemont-2025.7.1.finder.__path_hook__', '/opt/venv/lib/python3.12/site-packages/setuptools/_vendor']


True

In [5]:
# result = forestry.run_eligibility(config=config, use_gee=True)

# # Original validated points (before ML split)
# training_info = result['intermediate_results']['training_points_info']
# print(f"Original validated points: {training_info['num_points_after_validation']}")

# # Actual ML training points (after split)
# ml_training = result['intermediate_results']['ml_training_points']
# print(f"ML training points: {ml_training.size().getInfo()}")

# # Actual ML validation points (after split) - for confusion matrix
# ml_validation = result['intermediate_results']['ml_validation_points']
# print(f"ML validation points: {ml_validation.size().getInfo()}")

# # Use validation points for confusion matrix
# # lc.matrix_confusion(
# #     result['intermediate_results']['selected_image_lc'],
# #     ml_validation,
# #     result['intermediate_results']['algo_ml_selected']
# # )

In [6]:
from forestry_carbon_arr.example_usage import main
result = main()

Using example configuration. Adjust paths in config before running!
Starting Forestry ARR Eligibility analysis...
✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully
selecting Sentinel images
processing AVI
processing BSI
processing SI
Normalizing to 100 AVI
Normalizing to 100 AVI
Normalizing to 100 BSI
Normalizing to 100 SI
Combining AVI AND BSI
no thermal band, choosing Sentinel images
Processing means center of AVI_BSI please wait
Now we proceed to the PCA of Vegetation density
Success get the PCA normalized of VD => SVI
Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2
finish processing PCA, the result: FCD1_1 and FCD2_1 please continue
snic list bands: ['red_mean', 'green_mean', 'blue_mean', 'nir_mean', 'ndwi_mean', 'msavi2_mean', 'MTVI2_mean', 'NDVI_mean', 'VARI_mean', 'FCD1_1_mean', 'FCD2_1_mean', 'area', 'clusters_min', 'width', 'height']
Adding the map of Forest, FCD >= 45% and mask only if not wate

In [7]:
#Displaying the vectors
from wfs_manager import WFSManager

# result
AOI = result['config']['AOI']
training_points_ee = result['intermediate_results']['training_points_info']['training_points_ee']

wfs = WFSManager(fastapi_url="http://fastapi:8000", wfs_base_url="http://localhost:8001")
wfs.addLayer(training_points_ee, "Training Points Dynamic")
wfs.addLayer(AOI, "AOI Boundary")
wfs.publish()


{'status': 'success',
 'total_layers': 2,
 'successful_layers': 2,
 'failed_layers': 0,
 'layers': {'training_points_dynamic': {'status': 'success',
   'layer_name': 'training_points_dynamic',
   'fc_url': 'http://fastapi:8000/fc/training_points_dynamic',
   'wfs_url': 'http://localhost:8001/wfs?service=WFS&version=1.1.0&request=GetFeature&typename=training_points_dynamic',
   'feature_count': 200,
   'response': {'message': "FeatureCollection 'training_points_dynamic' created successfully",
    'name': 'training_points_dynamic',
    'type': 'FeatureCollection',
    'count': 200}},
  'aoi_boundary': {'status': 'success',
   'layer_name': 'aoi_boundary',
   'fc_url': 'http://fastapi:8000/fc/aoi_boundary',
   'wfs_url': 'http://localhost:8001/wfs?service=WFS&version=1.1.0&request=GetFeature&typename=aoi_boundary',
   'feature_count': 1,
   'response': {'message': "FeatureCollection 'aoi_boundary' created successfully",
    'name': 'aoi_boundary',
    'type': 'FeatureCollection',
    'cou

In [8]:
# unrelated to the project by for testing -->
import geemap
import os 
from gee_integration import add_tms_layer_to_mapstore
from gee_lib.osi.utils import generate_map_id

new_aoi_test = geemap.shp_to_ee('00_input/concession.shp')
wfs.addLayer(new_aoi_test, "new AOI Boundary")
wfs.publish()

path_cog =  os.getenv('GCS_PATH')
# cog gcs to ee image

drone_cog = geemap.load_GeoTIFF(path_cog)
### #type(drone_cog)

map_layers = generate_map_id({'drone_cog': {}}, { 'drone_cog': drone_cog})['map_layers']
map_layers

drone_info = map_layers['drone_cog']


### #!cat /root/.config/gcloud/application_default_credentials.json
### #re add
### result = list_layers_to_wmts(map_layers, AOI)
### #result
### #Add a TMS layer


drone_name = list(map_layers.keys())[0]
drone_layer = map_layers[drone_name]
tms_result = add_tms_layer_to_mapstore(
    layer_name=drone_name,
    layer_url=drone_info.get('tile_url', ''),
    use_fastapi_proxy=False, # change this to True if you want to use the fastapi proxy
    fastapi_pub_url="http://localhost:8001"
)

drone_layer


INFO:wfs_manager:Added vector layer: new AOI Boundary -> new_aoi_boundary
INFO:wfs_manager:Publishing 3 vector layers to WFS...
INFO:wfs_manager:✅ Published layer 'training_points_dynamic': FeatureCollection 'training_points_dynamic' created successfully
INFO:wfs_manager:✅ Published layer 'aoi_boundary': FeatureCollection 'aoi_boundary' created successfully
INFO:wfs_manager:✅ Published layer 'new_aoi_boundary': FeatureCollection 'new_aoi_boundary' created successfully
INFO:wfs_manager:WFS publishing completed: success
INFO:wfs_manager:  Successful: 3
INFO:wfs_manager:  Failed: 0


Generating GEE Map IDs...


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Adding TMS layer: drone_cog
INFO:gee_integration:✅ TMS layer added successfully
INFO:gee_integration:   Service ID: gee_tms_drone_cog
INFO:gee_integration:   Layer: Drone Cog
INFO:gee_integration:   URL: https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/3ac3cbd91d6b7b4e199b26475b8b6543-eddc77cc32dd91ea518041d62bcdc76f/tiles/{z}/{x}/{y}


{'tile_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/3ac3cbd91d6b7b4e199b26475b8b6543-eddc77cc32dd91ea518041d62bcdc76f/tiles/{z}/{x}/{y}',
 'name': 'Drone Cog',
 'description': 'DRONE_COG visualization from GEE analysis',
 'vis_params': {}}

In [9]:
result

{'final_zone': <ee.image.Image at 0x7f23db323800>,
 'intermediate_results': {'image_mosaick': <ee.image.Image at 0x7f23db497290>,
  'FCD1_1': <ee.image.Image at 0x7f23db400bc0>,
  'FCD2_1': <ee.image.Image at 0x7f23db4036b0>,
  'image_norm_with_spectral_indices_FCD': <ee.image.Image at 0x7f23dbb19c70>,
  'training_points_info': {'training_points_ee': <ee.featurecollection.FeatureCollection at 0x7f23db4de540>,
   'num_points_before_validation': 200,
   'num_points_after_validation': 200,
   'unique_classes': [1, 2, 3, 4, 9]},
  'ml_training_points': <ee.featurecollection.FeatureCollection at 0x7f23dbb48380>,
  'ml_validation_points': <ee.featurecollection.FeatureCollection at 0x7f23dbb485c0>,
  'classifier_results': {'training_points': <ee.featurecollection.FeatureCollection at 0x7f23dbb48380>,
   'validation_points': <ee.featurecollection.FeatureCollection at 0x7f23dbb485c0>,
   'training_input_variables': <ee.featurecollection.FeatureCollection at 0x7f23dbb4a780>,
   'validation_input

In [10]:
result['visualization_params']['FCD1_1']

{'min': 0, 'max': 80, 'palette': ['ff4c16', 'ffd96c', '39a71d']}

In [11]:
## displaying the rasters
### TRY the new Approach, wmts manager, make the simpler line for adding layer based on project
from wmts_manager import WMTSManager

project_name = result['config']['project_name']
image_mosaick = result['intermediate_results']['image_mosaick']
vis_param = result['visualization_params']['mosaic']
mosaic_name = result['intermediate_results']['layer_names']['image_mosaick']
FCD1_1 = result['intermediate_results']['FCD1_1']
fcd_visparams = result['visualization_params']['FCD1_1']
# fcd_visparams = {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}
fcd1_1_layer_name = result['intermediate_results']['layer_names']['FCD1_1']
FCD2_1 = result['intermediate_results']['FCD2_1']
fcd2_1_layer_name = result['intermediate_results']['layer_names']['FCD2_1']
treeLossYear = result['intermediate_results']['hansen_results']['treeLossYear']
treeLoss = result['intermediate_results']['hansen_results']['treeLoss']
minLoss = result['intermediate_results']['hansen_results']['minLoss']


wmts = WMTSManager(project_name=project_name, aoi=AOI.geometry()) # equal to the defining map properties 
wmts.addLayer(image_mosaick, vis_param, mosaic_name)
wmts.addLayer(FCD1_1, fcd_visparams, fcd1_1_layer_name)
wmts.addLayer(FCD2_1, fcd_visparams, fcd2_1_layer_name)
wmts.addLayer(treeLossYear.randomVisualizer(), {}, f'treeLossYear')
wmts.addLayer(treeLoss.randomVisualizer(), {}, f'treeLoss')
wmts.addLayer(minLoss.randomVisualizer(), {}, f'minLoss')
wmts.publish() # execution to the WMTS server

INFO:wmts_manager:WMTSManager initialized for project: belaban_rayak
INFO:wmts_manager:Added layer: Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor
INFO:wmts_manager:Added layer: FCD1_1_belaban_rayak
INFO:wmts_manager:Added layer: FCD2_1_belaban_rayak
INFO:wmts_manager:Added layer: treeLossYear
INFO:wmts_manager:Added layer: treeLoss
INFO:wmts_manager:Added layer: minLoss
INFO:wmts_manager:Generating map IDs for 6 layers...


Generating GEE Map IDs...
✅ Centroid calculated successfully with error margin 1


INFO:wmts_manager:AOI processed: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
INFO:wmts_manager:Publishing 6 layers to WMTS...
INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: belaban_rayak
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates


Calculated bbox from coordinates: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ AOI processed successfully:
   - Center: [110.46306597258281, -1.803818629688135]
   - Area: Unknown
   - BBox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}


INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'FCD1_1_belaban_rayak': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'FCD2_1_belaban_rayak': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'treeLossYear': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'treeLoss': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'minLoss': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Registering with FastAPI: belaban_rayak_20251114_141501
INF

{'status': 'success',
 'project_id': 'belaban_rayak_20251114_141501',
 'project_name': 'belaban_rayak',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 6},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 6 proxy URLs',
  'proxy_urls': {'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'proxy_url': 'http://fastapi:8000/tiles/belaban_rayak_20251114_141501/Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/80302b3165e2c7d995fd0de1c7c5c231-1c2eb31038dff7365282c75e6ca08bbe/tiles/{z}/{x}/{y}',
    'layer_name': 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor',
    'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis'},
   'FCD1_1_belaban_rayak': {'proxy_url': 'http://fastapi:8000/tiles/belaban_rayak_20251114_141501/FCD1_1_belaban_rayak/{z}/{x}/{y}',

In [12]:
vis_param

{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5}

In [ ]:
FCD1_1.bandNames().getInfo()

In [ ]:
result

In [ ]:
## LC
rf = result['intermediate_results']['classifier_results']['classified_image_rf']
svm = result['intermediate_results']['classifier_results']['classified_image_svm']
gbm = result['intermediate_results']['classifier_results']['classified_image_gbm']
cart = result['intermediate_results']['classifier_results']['classified_image_cart']

rf_layer_name = result['intermediate_results']['layer_names']['land_cover_rf']
svm_layer_name = result['intermediate_results']['layer_names']['land_cover_svm']
gbm_layer_name = result['intermediate_results']['layer_names']['land_cover_gbm']
cart_layer_name = result['intermediate_results']['layer_names']['land_cover_cart']

vis_param_lc = result['visualization_params']['land_cover']

wmts.addLayer(rf, vis_param_lc, rf_layer_name)
wmts.addLayer(svm, vis_param_lc, svm_layer_name)
wmts.addLayer(gbm, vis_param_lc, gbm_layer_name)
wmts.addLayer(cart, vis_param_lc, cart_layer_name)

## zone
# Access FCD classified zone
fcd_classified_zone = result['intermediate_results']['fcd_classified_zone']

# Access zones for all algorithms
zones = result['intermediate_results']['zones_all_algorithms']
zone_rf = zones['rf']
zone_svm = zones['svm']
zone_gbm = zones['gbm']
zone_cart = zones['cart']

# Access layer names
zones_names = result['intermediate_results']['layer_names']
fcd_zone_name = zones_names['fcd_classified_zone']
rf_zone_name = zones_names['zone_rf']
svm_zone_name = zones_names['zone_svm']
gbm_zone_name = zones_names['zone_gbm']
cart_zone_name = zones_names['zone_cart']

fcd_classified_zone = result['intermediate_results']['fcd_classified_zone']
vis_params_zone = result['visualization_params']['zone']

wmts.addLayer(fcd_classified_zone, vis_params_zone, rf_zone_name)
wmts.addLayer(zone_rf, vis_params_zone, rf_zone_name)
wmts.addLayer(zone_svm, vis_params_zone, svm_zone_name)
wmts.addLayer(zone_gbm, vis_params_zone, gbm_zone_name)
wmts.addLayer(zone_cart, vis_params_zone, cart_zone_name)

# re-overlay the data for zoning from the selected method if they give the best metric, and when we check visually the land cover map make sense, also FCD approach is already there
final_zone = result['final_zone']
final_zone_layer_name = result['intermediate_results']['layer_names']['final_zone']


wmts.addLayer(final_zone, vis_params_zone, final_zone_layer_name)
wmts.publish()

In [ ]:
# from google.auth import default
# creds, project = default()

In [ ]:
# !gcloud projects list
# !gcloud config list
# !gcloud auth list


In [ ]:
# !earthengine authenticate

In [ ]:
# !gcloud config set project ee-iwansetiawan

In [ ]:
# !pip install earthengine-api --upgrade


In [ ]:
# import os
# import ee
# from gee_lib.osi.auth import initialize_gee, check_gee_initialization

# # ee.Initialize(project=os.getenv('GOOGLE_CLOUD_PROJECT'))
# # ee.Initialize(project='ee-iwansetiawan')

# # Initialize GEE
# # authenticate = initialize_gee(project_id=os.getenv('GOOGLE_CLOUD_PROJECT'), use_service_account=False)
# authenticate = initialize_gee(project_id=os.getenv('GOOGLE_CLOUD_PROJECT'),use_service_account=True)

# # # Verify it's working
# # if check_gee_initialization():
# #     print("Ready to use GEE!")

In [ ]:
# # !gcloud --version

# # os.getenv('GOOGLE_CLOUD_PROJECT')

# from google.cloud import storage
# client = storage.Client()
# print(client.project)


In [ ]:
# !gcloud.storage.buckets.list

In [ ]:
# import json
# import os

# ### need improvement on how to manage this config, we also in the config the path to the static file
# # maybe we need to not put the path in the config.json, don't know yet
# # Move to the parent directory of the current script
# parent_dir = os.getcwd()

# # Construct the absolute path to the JSON file in the 'input' folder
# json_path= os.path.join(parent_dir, '00_input', 'balaban_conf.json')

# # Read and load the JSON data from the file
# with open(json_path, 'r') as file:
#     config = json.load(file)

# print('config---> ',config)

In [ ]:
# ## import modules
# import osi
# import pandas as pd
# import geemap

# from osi.utils.main import validate_aoi
# # convert the modules for image collection (cloudless masking, compositing, reducer etc)
# from osi.image_collection.main import ImageCollection
# from osi.spectral_indices.spectral_analysis import SpectralAnalysis
# from osi.spectral_indices.utils import normalization_100
# from osi.hansen.historical_loss import HansenHistorical
# from osi.classifying.assign_zone import AssignClassZone
# from osi.legends.utils import convert_to_legend_items
# from osi.legends.main import LegendsBuilder
# from osi.obia.main import OBIASegmentation
# from osi.ml.main import LandcoverML
# from osi.fcd.main_fcd import FCDCalc
# from osi.pca.pca_gee import PCA
# from osi.hansen.historical_loss import HansenHistorical
# from osi.classifying.assign_zone import AssignClassZone
# from osi.arcpy.utils import safe_get_data_source


# AOIt_shp_plot = geemap.shp_to_ee(config['AOI_path'])
# crs_input = config['crs_input']
# I_satellite = config['I_satellite']
# project_name = config['project_name']

# start_date = config['date_start_end'][0]
# end_date = config['date_start_end'][1]

# layer_name_image_mosaick = f'image_mosaick_result_ee_{project_name}'

# AOI = AOIt_shp_plot
# config['AOI'] = AOI

# ndwi_hi = 0.1
# if config['I_satellite'] == 'Landsat':
#     ndwi_hi = config['ndwi_hi_landsat']
# elif I_satellite == 'Sentinel':
#     ndwi_hi = config['ndwi_hi_sentinel']
# elif I_satellite == 'Planet':
#     ndwi_hi = config['ndwi_hi_planet']

# ### Masking and overlay and area helper Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
# OID = config['OID_field_name']
# AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
#     properties= [OID],
#     reducer= ee.Reducer.first()
# )

In [ ]:
# # now starting to do analysis
# # initiate instance class for the image collection and later mosaicking
# classInputCollection = ImageCollection(I_satellite=I_satellite,
#                                        AOI=AOI, 
#                                        date_start_end=config['date_start_end'], 
#                                        cloud_cover_threshold = config['cloud_cover_threshold'],
#                                        region=config['region'])

In [ ]:
# # run the method from image collection loaded, cloudless compositing until to image_mosaick
# image_mosaick = classInputCollection.image_mosaick()

In [ ]:
# from gee_lib.osi.utils import generate_map_id

# # Generate Map IDs
# ### Defining params visualization and layer name
# layer_name = f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor'

# if config['I_satellite'] == 'Planet':
#     # true color {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
#     # nir veg color {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5 }
#     vis_param = {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
# else:
#     vis_param = {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5}

# layers_data = {'project_name': 'Sentinel-2 Cloudless Composite Analysis'}


# print("✓ Map IDs generated for all layers")
# # Display tile URLs
# for layer_name, map_id_dict in generate_map_id({layer_name: vis_param}, {layer_name: image_mosaick})['all_mapid'].items():
#     tile_url = map_id_dict['tile_fetcher'].url_format
#     print(f"\n{layer_name.upper()}:")
#     print(f"  Tile URL: {tile_url}")



In [ ]:
# ## META data layers
# layers_data = {
#     'project_name': project_name,
# }

# # # Option 1: Simple tile URLs only
# # map_layers = {
# #     layer_name: map_ids[layer_name]['tile_fetcher'].url_format}

# # Option 2 with more detail combination for generating wmts
# map_layers = generate_map_id({layer_name: vis_param}, {layer_name: image_mosaick})['map_layers']
# map_layers



In [ ]:
# # import sys
# # sys.path.append('/usr/src/app/fastapi-gee-service')
# from gee_integration import process_gee_to_mapstore

# # Import the AOI fix function properly
# from gee_lib.osi.utils import process_aoi_geometry

# def list_layers_to_wmts(map_layers, AOI):
#     """
#     Convert map layers to WMTS format
    
#     Args:
#         map_layers: Dictionary of map layers

#     """
    
#     # Fix 1: Use correct map layers
#     print(f"\n🔍 Available map layers:")
#     print(f"   map_layers: {len(map_layers)}")

#     # Fix 2: Fix AOI coordinates using the existing function
#     print(f"\n🔧 Fixing AOI coordinates...")
#     AOI_geom = AOI.geometry()
#     aoi_info = process_aoi_geometry(AOI_geom)

#     print(f"✅ Fixed AOI Info:")
#     print(f"   Bbox: {aoi_info['bbox']}")
#     print(f"   Center: {aoi_info['center']}")

#     print("🚀 Processing GEE Analysis to MapStore...")
#     print(f"   Project: {layers_data['project_name']}")
#     print(f"   Layers: {len(map_layers)}")
#     print(f"   AOI: {aoi_info['bbox']}")

#     # Process the complete integration
#     result = process_gee_to_mapstore(
#         map_layers=map_layers,
#         project_name=layers_data['project_name'],
#         aoi_info=aoi_info,
#         clear_cache_first=True,
#         fastapi_url="http://fastapi:8000"  # Internal Docker network , this should be in localhost, but we can override as well
#     )
#     return result

# result = list_layers_to_wmts(map_layers, AOI) # EQUIVALENT TO MAP.ADDLAYER(RASTER)
# result

# ## EXAMPLE TO PUSH GEE TO GEOJSON FAST API and get WFS RESULT, (EQUIVALENT MAP.ADDLAYER (VECTOR))
# import requests
# import json

# # Example GeoJSON data
# geojson_data = AOI.getInfo() # important this ee.FeatureCollection should convert to geojson first
# name_aoi = 'gee_aoi_dynamic'

# link_fastapi = 'http://fastapi:8000/fc/'+name_aoi #only from container, not from localhost

# # Push to API
# response = requests.post(
#     link_fastapi, # running in the notebook for now
#     json=geojson_data
# )

# print('link_localhost: ', 'http://localhost:8001/fc/'+name_aoi)




In [ ]:
# !cat /usr/src/app/mapstore/config/localConfig.json

In [ ]:
# # Test the new TMS management system
# from gee_integration import add_tms_layer_to_mapstore, list_gee_tms_layers, clear_all_gee_tms_layers # still needed


# print("Adding TMS layer with Direct GEE TILES...")
# # Get the first layer as an example
# first_layer_name = list(map_layers.keys())[0]
# first_layer_info = map_layers[first_layer_name]

# # Add a TMS layer
# tms_result = add_tms_layer_to_mapstore(
#     layer_name=first_layer_name,
#     layer_url=first_layer_info.get('tile_url', ''),
#     use_fastapi_proxy=False, # change this to True if you want to use the fastapi proxy
#     fastapi_pub_url="http://localhost:8001"
# )

# print(f"\n📊 TMS Proxy Result:")
# print(f"   Status: {tms_result['status']}")
# if tms_result['status'] == 'success':
#     print(f"   Service ID: {tms_result['service_id']}")
#     print(f"   Layer Title: {tms_result['layer_title']}")
#     print(f"   TMS URL: {tms_result['tms_url']}")
#     print(f"   Uses FastAPI Proxy: {tms_result['use_fastapi_proxy']}")


In [ ]:
# list_gee_tms_layers()
# clear_all_gee_tms_layers()

In [ ]:
# import geopandas as gpd

# ### validate the input is correct
# ##AOI
# AOI_shp_path = config['AOI_path']
# AOI_shp = gpd.GeoDataFrame.from_file(AOI_shp_path)
# AOI_shp.head()

# # make sure the related id is exist in the AOI_shp
# id_name = config['OID_field_name']
# if id_name not in AOI_shp.columns:
#     raise ValueError(f"The field {id_name} does not exist in the AOI shapefile.")

# # check if the AOI_shp is in the correct crs
# if AOI_shp.crs != config['crs_input']:
#     AOI_shp = AOI_shp.to_crs(config['crs_input'])



In [ ]:
# ## validate the training points (this notebook assume the training points is already exist in the project)
# input_training_points_path = config['input_training']
# training_points = gpd.GeoDataFrame.from_file(input_training_points_path)

# # check if the training points is in the correct crs
# if training_points.crs != config['crs_input']:
#     training_points = training_points.to_crs(config['crs_input'])

# print('before_validation: ',training_points.shape)
# # Function to check if a value is an integer
# def is_integer(value):
#     return isinstance(value, int)

# # Filter out non-integer values in the 'code_lu' column
# training_points['code_lu'] = training_points['code_lu'].apply(lambda x: x if is_integer(x) else None)

# print('after validation: ',training_points.shape)
# training_points.head()


In [ ]:
## we can add the training point to the map (WFS) ;)
# training_points_ee = geemap.geopandas_to_ee(training_points)

In [ ]:
# # Example GeoJSON data, adding again above (points, not only AOI polygon)
# geojson_data = training_points_ee.getInfo() # important this ee.FeatureCollection should convert to geojson first
# name_aoi = 'training_points_dynamic'

# link_fastapi = 'http://fastapi:8000/fc/'+name_aoi #only from container, not from localhost

# # Push to API
# response = requests.post(
#     link_fastapi, # running in the notebook for now
#     json=geojson_data
# )

In [ ]:
# pallette_class_segment = {
#             '1': '#83ff5a',  # forest_trees (1)
#             '2': '#ffe3b3',  # shrubland (2)
#             '3': '#ffff33',  # grassland (3)
#             '4': '#f89696',  # openland (4)
#             '5': '#1900ff',  # waterbody_wet_area (5)
#             '6': '#e6e6fa',  # plantation (6)
#             '7': '#FFFFFF',   # gray_infrastructure (7)
#             '8': '#4B0082',  # oil_palm (8) - Dark Purple
#             '9': '#8B4513',  # cropland (9) - Brown
#             '10': '#87CEEB',  # waterbody (10) - Light Blue
#             '11': '#2F4F4F',  # wetlands (11) - Dark Teal
#             '12': '#ADFF2F',  # forest_trees_regrowth (12) - Light Green
#             '13': '#8B0000',  # historical_treeloss_10years (13) - Dark Red
#             '14': '#DAA520'   # paddy_irrigated (14) - Golden Yellow

#             }

# training_points['code_lu'].unique()

In [ ]:
# [f.name for f in map.listLayers()]
# [f.name for f in arc_ops.list_layers]

In [ ]:
# classImageSpectral = SpectralAnalysis(image_mosaick,config)
# class_FCD_run = FCDCalc(config).fcd_calc()
# FCD1_1 = class_FCD_run['FCD1_1']
# FCD2_1 = class_FCD_run['FCD2_1']

# # FCD1_1_arcgis_layer = arc_ops.adding_ee_to_arcgisPro(FCD1_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']},
# #                                                    f'FCD1_1_{project_name}')

# fcd_visparams = {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}
# fcd1_1_layer_name = f'FCD1_1_{project_name}'
# fcd2_1_layer_name = f'FCD2_1_{project_name}'

# # FCD2_1_arcgis_layer = arc_ops.adding_ee_to_arcgisPro(FCD2_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']},
# #                                                    f'FCD2_1_{project_name}')

# map_layers = generate_map_id({layer_name: vis_param, fcd1_1_layer_name: fcd_visparams, fcd2_1_layer_name: fcd_visparams}, 
#                              {layer_name: image_mosaick, fcd1_1_layer_name: FCD1_1, fcd2_1_layer_name: FCD2_1})['map_layers']

# print('finish processing PCA please continue')

In [ ]:
# map_layers

In [ ]:
# result = list_layers_to_wmts(map_layers, AOI) # EQUIVALENT TO MAP.ADDLAYER(RASTER)

In [ ]:
# ## starting to look at the historical data (Hansen - global foresty watch)
# hansen_class = HansenHistorical(config)
# run_hansen = hansen_class.initiate_tcl()
# LastImageLandsat, treeLossYear, minLoss, ForestArea2000Hansen, gfc =  \
#                                  run_hansen['LastImageLandsat'], \
#                                  run_hansen['treeLossYear'], \
#                                  run_hansen['minLoss'], \
#                                  run_hansen['ForestArea2000Hansen'], \
#                                  run_hansen['gfc']

In [ ]:
# ## if we want to add the hansen layer to the map
# tree_loss_year = treeLossYear.randomVisualizer()

# # arc_ops.adding_ee_to_arcgisPro(treeLossYear.randomVisualizer(), {},
# #                                                    f'treeLossYear')

# treeLoss = treeLossYear.gte(0).And(treeLossYear.lte(23)).selfMask()
# # arc_ops.adding_ee_to_arcgisPro(treeLoss.randomVisualizer(), {},
# #                                                    f'treeLoss')

# #Canopy cover percentage (e.g. 30%), for Indonesia
# cc = ee.Number(30)

# #Minimum forest area in pixels (e.g. 3 pixels, ~ 0.27 ha in this example).
# pixels = ee.Number(3)

# #Minimum mapping area for tree loss (usually same as the minimum forest area).
# lossPixels = pixels

# canopyCover = gfc.select(['treecover2000'])
# canopyCoverThreshold = canopyCover.gte(cc).selfMask()

# #Use connectedPixelCount() to get contiguous area.
# contArea = canopyCoverThreshold.connectedPixelCount()
# #Apply the minimum area requirement.
# minArea = contArea.gte(pixels).selfMask()

# treecoverLoss = minArea.And(treeLoss).rename(f'lossfrom_0-23').selfMask()
        
# #Create connectedPixelCount() to get contiguous area.
# contLoss = treecoverLoss.connectedPixelCount()
# #Apply the minimum area requirement, and get the TCL data ---> minLoss - ACTUAL TCL AREA from Hansen since the year_start_loss
# minLoss = contLoss.gte(lossPixels).selfMask()


# # arc_ops.adding_ee_to_arcgisPro(minLoss.randomVisualizer(), {},
# #                                                    f'minLoss')

In [ ]:
# ### TRY the new Approach, wmts manager, make the simpler line for adding layer based on project
# from wmts_manager import WMTSManager

# wmts = WMTSManager(project_name=project_name, aoi=AOI.geometry()) # equal to the defining map properties 
# wmts.addLayer(image_mosaick, vis_param, layer_name)
# wmts.addLayer(FCD1_1, fcd_visparams, fcd1_1_layer_name)
# wmts.addLayer(FCD2_1, fcd_visparams, fcd2_1_layer_name)
# wmts.addLayer(treeLossYear.randomVisualizer(), {}, f'treeLossYear')
# wmts.addLayer(treeLoss.randomVisualizer(), {}, f'treeLoss')
# wmts.addLayer(minLoss.randomVisualizer(), {}, f'minLoss')
# result = wmts.publish() # execution to the WMTS server

In [ ]:
# from wfs_manager import WFSManager

# wfs = WFSManager(fastapi_url="http://fastapi:8000", wfs_base_url="http://localhost:8001")
# wfs.addLayer(training_points_ee, "Training Points Dynamic")
# wfs.addLayer(AOI, "AOI Boundary")
# result = wfs.publish()

In [ ]:
# # NEW (Clean and Flexible):
# from osi.utils.main import generate_map_id_list, create_layer_info

# # Define your layers in a clean list
# layers = [
#     create_layer_info(layer_name, image_mosaick, vis_param, "Sentinel Mosaicked Image"),
#     create_layer_info(fcd1_1_layer_name, FCD1_1, fcd_visparams, "Forest Cover Density 1"),
#     create_layer_info(fcd2_1_layer_name, FCD2_1, fcd_visparams, "Forest Cover Density 2")
# ]

# # Generate Map IDs
# result = generate_map_id_list(layers)
# map_layers = result['map_layers']
# map_layers

In [ ]:
# # arc_ops.list_layers = map.listLayers()
# # arc_ops.list_source_layers_in_map = [safe_get_data_source(layer) for layer in arc_ops.list_layers]

# config['AOI_img'] = AOI_img

# class_assigning_fcd =  AssignClassZone(config, FCD1_1=FCD1_1, FCD2_1=FCD2_1)
# list_images_classified = class_assigning_fcd.assigning_fcd_class(gfc, minLoss)

# fcd_classified_zone = list_images_classified['all_zone']

# vis_params_fcd_classified = class_assigning_fcd.vis_param_merged
# # Convert the dictionary to the LEGEND_ITEMS format
# legend_items = convert_to_legend_items(class_assigning_fcd.legend_class)

# # fcd_classified_zone_arcgis_layer = arc_ops.adding_ee_to_arcgisPro(fcd_classified_zone, vis_params_fcd_classified,
# #                                                    f'FCD_classified_zone_{project_name}')
# wmts.addLayer(fcd_classified_zone, vis_params_fcd_classified, f'FCD_classified_zone_{project_name}')
# result = wmts.publish()


In [ ]:
# ## ADDING DIRECTLY SPECTRAL INDICES
# # classImageSpectral
# pca_scale = classImageSpectral.pca_scale #pca_scale is spatial resolution. eg planet: 5
# ndwi_image = classImageSpectral.NDWI_func()
# msavi2_image = classImageSpectral.MSAVI2_func()
# mtvi2_image = classImageSpectral.MTVI2_func()
# ndvi_image = classImageSpectral.NDVI_func()
# vari_image = classImageSpectral.VARI_func()

# image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari = (
#     image_mosaick
#     .addBands(ndwi_image)
#     .addBands(msavi2_image)
#     .addBands(mtvi2_image)
#     .addBands(ndvi_image)
#     .addBands(vari_image)
# )

# red_norm = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select(['red']), pca_scale=pca_scale, AOI=AOI)
# green_norm = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select(['green']), pca_scale=pca_scale, AOI=AOI)
# blue_norm = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select(['blue']), pca_scale=pca_scale, AOI=AOI)
# nir_norm = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select(['nir']), pca_scale=pca_scale, AOI=AOI)

# image_norm = red_norm.addBands(green_norm).addBands(blue_norm).addBands(nir_norm)

# image_norm_ndvi = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('NDVI'), pca_scale=pca_scale, AOI=AOI)
# image_norm_ndwi = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('ndwi'), pca_scale=pca_scale, AOI=AOI)
# image_norm_msavi2 = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('msavi2'), pca_scale=pca_scale, AOI=AOI)
# image_norm_mtvi2 = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('MTVI2'), pca_scale=pca_scale, AOI=AOI)
# image_norm_vari = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('VARI'), pca_scale=pca_scale, AOI=AOI)

# # red_norm.bandNames().getInfo()
# image_norm_with_spectral_indices = image_norm.addBands(image_norm_ndvi).addBands(image_norm_ndwi).addBands(image_norm_msavi2).addBands(image_norm_mtvi2).addBands(image_norm_vari)
# image_norm_with_spectral_indices_FCD = image_norm_with_spectral_indices.addBands(FCD2_1.select('FCD').rename('FCD2_1')).addBands(FCD1_1.select('FCD').rename('FCD1_1'))

     

In [ ]:
# obia = OBIASegmentation(config=config, image=image_norm_with_spectral_indices_FCD, pca_scale=pca_scale) #pca_scale basically is spatial resolution e.g planet: 5
# clusters = obia.SNIC_cluster()['clusters']
# object_properties_image = obia.summarize_cluster(is_include_std = False)

# lc = LandcoverML(config=config,
#                  input_image = image_norm_with_spectral_indices_FCD,
#                 cluster_properties=object_properties_image,
#                 pca_scale = pca_scale)

# classifier = lc.run_classifier()

# legend_lc = lc.lc_legend_param()
# vis_param_lc = legend_lc['vis_param_lc']

# legend_lc = legend_lc['legend_class']
# # Convert the dictionary to the LEGEND_ITEMS format
# legend_items_lc = convert_to_legend_items(legend_lc)

# training_points = classifier['training_points']
# validation_points = classifier['validation_points']

# rf = classifier['classified_image_rf']
# svm = classifier['classified_image_svm']
# gbm = classifier['classified_image_gbm']
# cart = classifier['classified_image_cart']

# # fcd lc, 5 classes only, just nice to know
# fcd_lc = list_images_classified['fcd_class_lc_image']
# fcd_lc_vs = list_images_classified['vis_param_segment_lc']

In [ ]:
# image_mosaick_all_bands = image_mosaick.addBands([FCD2_1.select('FCD').rename('FCD2_1'), FCD1_1.select('FCD').rename('FCD1_1')])

# ## ADDING DIRECTLY SPECTRAL INDICES
# # classImageSpectral
# pca_scale = classImageSpectral.pca_scale
# ndwi_image = classImageSpectral.NDWI_func()
# msavi2_image = classImageSpectral.MSAVI2_func()
# mtvi2_image = classImageSpectral.MTVI2_func()
# ndvi_image = classImageSpectral.NDVI_func()
# vari_image = classImageSpectral.VARI_func()

# image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari = (
#     image_mosaick_all_bands
#     .addBands(ndwi_image)
#     .addBands(msavi2_image)
#     .addBands(mtvi2_image)
#     .addBands(ndvi_image)
#     .addBands(vari_image)
# )

# red_norm = normalization_100(image_mosaick.select(['red']), pca_scale=pca_scale, AOI=AOI)
# green_norm = normalization_100(image_mosaick.select(['green']), pca_scale=pca_scale, AOI=AOI)
# blue_norm = normalization_100(image_mosaick.select(['blue']), pca_scale=pca_scale, AOI=AOI)
# nir_norm = normalization_100(image_mosaick.select(['nir']), pca_scale=pca_scale, AOI=AOI)

# image_norm = red_norm.addBands(green_norm).addBands(blue_norm).addBands(nir_norm)

# image_norm_ndvi = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('NDVI'), pca_scale=pca_scale, AOI=AOI)
# image_norm_ndwi = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('ndwi'), pca_scale=pca_scale, AOI=AOI)
# image_norm_msavi2 = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('msavi2'), pca_scale=pca_scale, AOI=AOI)
# image_norm_mtvi2 = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('MTVI2'), pca_scale=pca_scale, AOI=AOI)
# image_norm_vari = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('VARI'), pca_scale=pca_scale, AOI=AOI)

# # red_norm.bandNames().getInfo()
# image_norm_with_spectral_indices = image_norm.addBands(image_norm_ndvi).addBands(image_norm_ndwi).addBands(image_norm_msavi2).addBands(image_norm_mtvi2).addBands(image_norm_vari)
# image_norm_with_spectral_indices_FCD = image_norm_with_spectral_indices.addBands(FCD2_1.select('FCD').rename('FCD2_1')).addBands(FCD1_1.select('FCD').rename('FCD1_1'))

In [ ]:
# image_mosaick_all_bands = image_mosaick.addBands([FCD2_1.select('FCD').rename('FCD2_1'), FCD1_1.select('FCD').rename('FCD1_1')])

# ## ADDING DIRECTLY SPECTRAL INDICES
# # classImageSpectral
# pca_scale = classImageSpectral.pca_scale
# ndwi_image = classImageSpectral.NDWI_func()
# msavi2_image = classImageSpectral.MSAVI2_func()
# mtvi2_image = classImageSpectral.MTVI2_func()
# ndvi_image = classImageSpectral.NDVI_func()
# vari_image = classImageSpectral.VARI_func()

# image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari = (
#     image_mosaick_all_bands
#     .addBands(ndwi_image)
#     .addBands(msavi2_image)
#     .addBands(mtvi2_image)
#     .addBands(ndvi_image)
#     .addBands(vari_image)
# )

# red_norm = normalization_100(image_mosaick.select(['red']), pca_scale=pca_scale, AOI=AOI)
# green_norm = normalization_100(image_mosaick.select(['green']), pca_scale=pca_scale, AOI=AOI)
# blue_norm = normalization_100(image_mosaick.select(['blue']), pca_scale=pca_scale, AOI=AOI)
# nir_norm = normalization_100(image_mosaick.select(['nir']), pca_scale=pca_scale, AOI=AOI)

# image_norm = red_norm.addBands(green_norm).addBands(blue_norm).addBands(nir_norm)

# image_norm_ndvi = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('NDVI'), pca_scale=pca_scale, AOI=AOI)
# image_norm_ndwi = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('ndwi'), pca_scale=pca_scale, AOI=AOI)
# image_norm_msavi2 = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('msavi2'), pca_scale=pca_scale, AOI=AOI)
# image_norm_mtvi2 = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('MTVI2'), pca_scale=pca_scale, AOI=AOI)
# image_norm_vari = normalization_100(image_mosaick_ndvi_ndwi_msavi2_mtvi2_vari.select('VARI'), pca_scale=pca_scale, AOI=AOI)

# # red_norm.bandNames().getInfo()
# image_norm_with_spectral_indices = image_norm.addBands(image_norm_ndvi).addBands(image_norm_ndwi).addBands(image_norm_msavi2).addBands(image_norm_mtvi2).addBands(image_norm_vari)
# image_norm_with_spectral_indices_FCD = image_norm_with_spectral_indices.addBands(FCD2_1.select('FCD').rename('FCD2_1')).addBands(FCD1_1.select('FCD').rename('FCD1_1'))
     

In [ ]:
# from datetime import datetime

# obia = OBIASegmentation(config=config, image=image_norm_with_spectral_indices_FCD, pca_scale=pca_scale) #pca_scale basically is spatial resolution e.g planet: 5
# clusters = obia.SNIC_cluster()['clusters']
# object_properties_image = obia.summarize_cluster(is_include_std = False)
# # make sure has all the same type of data in all bands, for exporting purpose
# object_properties_image = object_properties_image.clip(AOI).toFloat()

# config["date_analyzed"] = datetime.now().strftime("%Y-%m-%d")

# lc = LandcoverML(config=config,
#                  input_image = image_norm_with_spectral_indices_FCD,
#                 cluster_properties=object_properties_image,
#                  num_class=5, # make sure this one is align with total type landcover stratification, for a sample creation, but not for using the existing input training
#                 pca_scale = pca_scale)

In [ ]:
import numpy as np

create_training_gee = False

#create stratified random sampling based on K-means classes
# create_training_gee
if create_training_gee:
    print('yes')
    
    ########### SAMPLE NUMBER CREATION BASED ON https://docs.google.com/spreadsheets/d/1J8MEi4IDn6faok6UUn9L64T61yWk0D4q/edit?gid=1919918133#gid=1919918133
#     # example
#     strata_area_based_kmeans = {
#     'Forest': 100,
#     'Shrub': 100,
#     'Grass': 100,
#     'Crop': 100,
#     'Water': 100
#     }
    
    # try with K-means input
    random_samples_creation = lc.stratified_random_creation()
    df_sample_n = random_samples_creation['df_sample_n']
    stratified_training = random_samples_creation['stratified_training']
       
    # Export the samples to a CSV file in Google Drive
    export_stratified_point = ee.batch.Export.table.toDrive(
        collection=stratified_training,
        description=f'Stratified_Random_Samples_{project_name}',
        folder=f'lu_input_{project_name}',
        fileFormat='SHP'
    )

    # Start the export task
    export_stratified_point.start()

    # Monitor the task status
    import time
    while export_stratified_point.active():
        print('Export task status:', export_stratified_point.status())
        time.sleep(10)

    print(f'Export task completed: Stratified_Random_Samples_{project_name}')
    
    # Open in gdrive in your computer
    location_stratified_exported = fr'G:\My Drive\lu_input_{project_name}\Stratified_Random_Samples_{project_name}.shp'
    print(location_stratified_exported) #make sure the location is correct (G: is connected by gdrive desktop!)
    
    # arcgis layer object
    training_points = map.addDataFromPath(location_stratified_exported)
    # training_points.name = 'change the name here'
    
    path_shp_input_training = training_points.dataSource
    
else:
    print('we will use the existing training_points or labelled')
    print(f'location of the training sample is in {config["input_training"]}')

In [ ]:
# arc_ops.list_layers = map.listLayers()
# arc_ops.list_source_layers_in_map = [safe_get_data_source(layer) for layer in arc_ops.list_layers]

# starting to do ML analysis
classifier = lc.run_classifier()

legend_lc = lc.lc_legend_param()
vis_param_lc = legend_lc['vis_param_lc']

training_points = classifier['training_points']
validation_points = classifier['validation_points']

rf = classifier['classified_image_rf']
svm = classifier['classified_image_svm']
gbm = classifier['classified_image_gbm']
cart = classifier['classified_image_cart']

# fcd lc, 5 classes only, just nice to know
fcd_lc = list_images_classified['fcd_class_lc_image']
fcd_lc_vs = list_images_classified['vis_param_segment_lc']
# fcd_lc_arc = arc_ops.adding_ee_to_arcgisPro(fcd_lc,fcd_lc_vs, 'fcd-method_lc_result')
# rf_arc = arc_ops.adding_ee_to_arcgisPro(rf,vis_param_lc,'Random_forest_lc_result')
# svm_arc = arc_ops.adding_ee_to_arcgisPro(svm,vis_param_lc,'SVM_lc_result')
# gbm_arc = arc_ops.adding_ee_to_arcgisPro(gbm,vis_param_lc,'GBM_lc_result')
# cart_arc = arc_ops.adding_ee_to_arcgisPro(cart,vis_param_lc,'CART_lc_result')

wmts.addLayer(rf, vis_param_lc, 'Random_forest_lc_result')
wmts.addLayer(svm, vis_param_lc, 'SVM_lc_result')
wmts.addLayer(gbm, vis_param_lc, 'GBM_lc_result')
wmts.addLayer(cart, vis_param_lc, 'CART_lc_result')
result = wmts.publish()


In [ ]:
output_dir = os.path.join(os.getcwd(),"01_output") # example we create output directory
os.makedirs(output_dir, exist_ok=True)
lc.matrix_confusion(fcd_lc,validation_points,'fcd',output_dir)
lc.matrix_confusion(rf, validation_points, 'rf', output_dir)
lc.matrix_confusion(svm, validation_points, 'svm', output_dir)
lc.matrix_confusion(gbm, validation_points, 'gbm', output_dir)
lc.matrix_confusion(cart, validation_points, 'cart', output_dir)

In [ ]:
algo_ml_selected = 'rf'
selected_image_lc = rf
if config['algo_ml_selected'] == 'rf':
    algo_ml_selected = 'rf'
    selected_image_lc = rf
elif config['algo_ml_selected'] == 'svm':
    algo_ml_selected = 'svm'
    selected_image_lc = svm
elif config['algo_ml_selected'] == 'gbm':
    algo_ml_selected = 'gbm'
    selected_image_lc = gbm
elif config['algo_ml_selected'] == 'cart':
    algo_ml_selected = 'cart'
    selected_image_lc = cart
print('algo_ml_selected: ',algo_ml_selected)

In [ ]:
# re-overlay the data for zoning from the selected method if they give the best metric, and when we check visually the land cover map make sense, also FCD approach is already there
image_for_zone = selected_image_lc

# comment this first, just check the LC above, then run the overlay zoning classification after
HighForestDense = list_images_classified['HighForestDense']

final_zone = class_assigning_fcd.assign_zone_ml(image_for_zone, minLoss,AOI_img, HighForestDense)
wmts.addLayer(final_zone, vis_params_fcd_classified, f'Final_zone_ML_{algo_ml_selected}_Hansen')
result = wmts.publish()

# arc_ops.adding_ee_to_arcgisPro(final_zone, vis_params_fcd_classified, f'Final_zone_ML_{algo_ml_selected}_Hansen')  # the naming probably will need to change, for some concistencies only so that you understand again later to read the codes